In [1]:
import tensorflow as tf
print(tf.__version__)

2.4.1


In [2]:
import os
import random
import pandas as pd
import numpy as np
import cv2
from keras.utils import np_utils

In [3]:
IMAGE_SIZE = 100
CLASS = ["angry","disgust","fear","happy","neutral","sad","surprise"]
train_dir = "images/train"
test_dir = "images/test"

In [4]:
angry_path = os.path.join(train_dir,CLASS[0])
os.listdir(angry_path)

['24283.jpg',
 '29810.jpg',
 '10242.jpg',
 '9535.jpg',
 '19369.jpg',
 '7046.jpg',
 '34333.jpg',
 '11102.jpg',
 '5967.jpg',
 '10894.jpg',
 '8930.jpg',
 '12049.jpg',
 '17696.jpg',
 '28811.jpg',
 '34944.jpg',
 '7187.jpg',
 '33542.jpg',
 '8423.jpg',
 '4853.jpg',
 '18065.jpg',
 '27516.jpg',
 '33682.jpg',
 '15700.jpg',
 '360.jpg',
 '7779.jpg',
 '1513.jpg',
 '27668.jpg',
 '28647.jpg',
 '25634.jpg',
 '16253.jpg',
 '35069.jpg',
 '12270.jpg',
 '28162.jpg',
 '31077.jpg',
 '18912.jpg',
 '29868.jpg',
 '5603.jpg',
 '28089.jpg',
 '4501.jpg',
 '30526.jpg',
 '16638.jpg',
 '5246.jpg',
 '5129.jpg',
 '33064.jpg',
 '16919.jpg',
 '3702.jpg',
 '19344.jpg',
 '6898.jpg',
 '19242.jpg',
 '26263.jpg',
 '30852.jpg',
 '14137.jpg',
 '32431.jpg',
 '29503.jpg',
 '18806.jpg',
 '31586.jpg',
 '29544.jpg',
 '19766.jpg',
 '4438.jpg',
 '6952.jpg',
 '1397.jpg',
 '28885.jpg',
 '13517.jpg',
 '11377.jpg',
 '32423.jpg',
 '26449.jpg',
 '26773.jpg',
 '25611.jpg',
 '7096.jpg',
 '30609.jpg',
 '1098.jpg',
 '29348.jpg',
 '15112.jpg',


In [5]:
training_data = []

In [6]:
def create_data(dir_name) :
    
    for cl in CLASS :
        cl_path = os.path.join(dir_name,cl)
        class_num = CLASS.index(cl)
        for img in os.listdir(cl_path) :
            try :
                img_array = cv2.imread(os.path.join(cl_path,img))
                img_array = cv2.resize(img_array,(IMAGE_SIZE,IMAGE_SIZE))
                gray_img  = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
                training_data.append([gray_img, class_num])
            except :
                pass
            
        random.shuffle(training_data)
        
    #return data_set
            
        

In [7]:
create_data(train_dir)
create_data(test_dir)

In [8]:
#shuffle the data
random.shuffle(training_data)
print(len(training_data))

35887


In [9]:
X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)

In [10]:
(np.array(X).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)).shape

(35887, 100, 100, 1)

In [10]:
X = np.array(X).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)
y = np.array(y)

X = X/255.0

In [11]:
num_classes = len(os.listdir(train_dir))
num_classes

7

In [12]:
X.shape[1:]

(100, 100, 1)

In [13]:
# convert class labels to one-hot encoding
y = np_utils.to_categorical(y, num_classes)

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

In [15]:
# building the model
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=X.shape[1:]))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(96, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(96, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(180, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(96, activation='relu'))
model.add(Dense(64, activation='relu'))

model.add(Dense(num_classes,activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 64)        640       
_________________________________________________________________
batch_normalization (BatchNo (None, 98, 98, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 49, 49, 96)        55392     
_________________________________________________________________
batch_normalization_1 (Batch (None, 49, 49, 96)        384       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 96)        8

In [17]:
history = model.fit(X, y, batch_size=32, epochs=10, validation_split=0.1)

Epoch 1/10
1010/1010 [==============================] - 808s 784ms/step - loss: 1.7031 - accuracy: 0.3288 - val_loss: 1.6809 - val_accuracy: 0.4054
Epoch 2/10
1010/1010 [==============================] - 815s 807ms/step - loss: 1.2333 - accuracy: 0.5304 - val_loss: 1.3290 - val_accuracy: 0.4940
Epoch 3/10
1010/1010 [==============================] - 780s 773ms/step - loss: 1.0753 - accuracy: 0.5971 - val_loss: 1.1804 - val_accuracy: 0.5592
Epoch 4/10
1010/1010 [==============================] - 783s 776ms/step - loss: 0.9230 - accuracy: 0.6555 - val_loss: 1.1660 - val_accuracy: 0.5584
Epoch 5/10
1010/1010 [==============================] - 802s 794ms/step - loss: 0.7765 - accuracy: 0.7118 - val_loss: 1.3244 - val_accuracy: 0.5564
Epoch 6/10
1010/1010 [==============================] - 778s 770ms/step - loss: 0.6260 - accuracy: 0.7714 - val_loss: 1.2104 - val_accuracy: 0.5921
Epoch 7/10
1010/1010 [==============================] - 824s 816ms/step - loss: 0.4741 - accuracy: 0.8319 - val_

In [18]:
model.save("models/model2.h5")